In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import string
import wikipedia

In [2]:
web_info = requests.get('https://en.wikipedia.org/wiki/List_of_documentary_films').text
soup = BeautifulSoup(web_info, 'lxml')
#print(soup.prettify())
list_table = soup.findAll('table', {'class': 'wikitable sortable'})

In [3]:
# create a list of all the names you think/know the section might be called
possibles = ['Plot','Synopsis','Plot synopsis','Plot summary', 
             'Story','Plotline','The Beginning','Summary',
            'Content','Premise', 'Overview', 'Details']

# sometimes those names have 'Edit' latched onto the end due to 
# user error on Wikipedia. In that case, it will be 'PlotEdit'
# so it's easiest just to make another list that acccounts for that
possibles_edit = [i + 'Edit' for i in possibles]
#then merge those two lists together
all_possibles = possibles + possibles_edit

In [ ]:
def get_doc_list_num():
    doc_list = {}

    tab = list_table[0].findAll('i')
    for i in range(len(tab)):
        for node in tab[i].findAll('a'):
            for idx in list(string.digits):
                if ''.join(node.findAll(text = True)).startswith(idx):
                    name = ''.join(node.findAll(text = True))
                    #links.append('https://www.wikipedia.org'+node.get('href'))

                    p = wikipedia.page(name).sections[0]
                    if p in all_possibles:
                        doc_list[name] = wikipedia.page(name).section(p).replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
                    else:
                        doc_list[name] = wikipedia.page(name).summary.replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')

    return doc_list

df = pd.DataFrame(list(get_doc_list_num().items()), columns = ['Title', 'Description'])

df.to_csv('doc_list/list_try.csv', header = True, index = False)

In [ ]:
def get_doc_list_alph(start):
    doc_list = {}
    
    tab = list_table[start].findAll('i')
    
    for i in range(len(tab)):
        for node in tab[i].findAll('a'):
            if ''.join(node.findAll(text = True)).startswith(string.ascii_uppercase[start - 1]):
                name = ''.join(node.findAll(text = True))
                #link = 'https://www.wikipedia.org'+node.get('href')
                wiki_name = ''.join([name, ' (film)'])
                
                try:
                    p = wikipedia.page(wiki_name).sections[0]
                    #print(wiki_name, p)
                    if p in all_possibles:
                        doc_list[name] = wikipedia.page(wiki_name).section(p).replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
                    else:
                        doc_list[name] = wikipedia.page(wiki_name).summary.replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
                except IndexError:
                    pass
                
                except wikipedia.exceptions.DisambiguationError:
                    #pass
                    print(wiki_name)
                except wikipedia.exceptions.PageError:
                    pass           
                    
    return doc_list

df = pd.DataFrame(list(get_doc_list_alph(7).items()), columns = ['Title', 'Description'])

df.to_csv('doc_list/list_g.csv', header = True, index = False)



#get_doc_list_alph(2)



In [ ]:
t = list_table[26].findAll('i')
n = []
for i in range(len(t)):
    for node in t[i].findAll('a'):
        if ''.join(node.findAll(text = True)).startswith('Z'):
            n.append(''.join(node.findAll(text = True)))
print(len(n))

In [ ]:
def get_l_list():
    
    l_list = ['Life on Earth', 'The Living Planet', 'The Trials of Life', 'Life in the Freezer',
        'The Private Life of Plants', 'The Life of Birds', 
        'The Life of Mammals', 'Life in the Undergrowth', 'Life in Cold Blood']

    doc_list = {}
    
    for j in l_list:
        wiki_name = ''.join([j, ' (film)'])
        #print(wiki_name)
        try:
            p = wikipedia.page(wiki_name).sections[0]
            #print(wiki_name, p)
            if p in all_possibles:
                doc_list[wiki_name.split('(')[0]] = wikipedia.page(wiki_name).section(p).replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
            else:
                doc_list[wiki_name.split('(')[0]] = wikipedia.page(wiki_name).summary.replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
        except IndexError:
            pass

        except wikipedia.exceptions.DisambiguationError:
            #pass
            print(wiki_name)
        except wikipedia.exceptions.PageError:
            pass           
                    
    return doc_list

df = pd.DataFrame(list(get_l_list().items()), columns = ['Title', 'Description'])

df.to_csv('doc_list/list_missl.csv', header = True, index = False)


In [ ]:
missing_names = ['Amarillo by Morning', 'Beyond Belief', 'Call + Response', 'Castro Street',
                'Dust to Dust: The Health Effects of 9/11 (film)',
                'Finders Keepers',
                'Midnight Movies: From the Margin to the Mainstream',
                'Million Calorie March: The Movie', 'No Place Like Home', 'The Outsider',
                'Operation Wedding: My parents & the plane hijacking', 'Primary',
                'Poetry in Motion', 'The Roosevelts (PBS)', 'Red String, The',
                'Rolling Papers (film)', 'South of the Border', 'Surplus']
print(len(missing_names))

In [4]:
d1 = {}
wiki_name = 'Amarillo by Morning (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
desc = soup.findAll('p')[0].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d1[wiki_name.split('(')[0]] = desc
d1

{'Amarillo by Morning ': 'amarillo by morning is a 1997 documentary short by spike jonze. while filming pro bullriders for a commercial at the national rodeo in the houston astrodome in houston, texas, spike befriended two suburban teenagers who aspired to be cowboys. the documentary chronicles an afternoon in their lives.'}

In [5]:
d2 = {}
wiki_name = 'Audience of One (2007 film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d2[wiki_name.split('(')[0]] = soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d2

{'Audience of One ': 'the film follows the story of a san francisco pentecostal minister richard gazowsky on his quest to shoot a groundbreaking fantasy film called gravity: the shadow of joseph (described by him as "star wars meets the ten commandments"). the film follows him and members of his church as they go through preproduction and fly to alberobello, italy, for initial shooting that turns out to be marred with difficulties. after returning home, gazowsky manages to arrange a lease of the treasure island film studio, but as their promised financing from german investors never materializes, they get evicted and eventually sued by the city of san francisco for not paying their rent.'}

In [6]:
d3 = {}
wiki_name = 'Beyond Belief (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d3[wiki_name.split('(')[0]] = soup.find('p').text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '').split('[')[0]
d3

{'Beyond Belief ': 'beyond belief is a feature documentary directed by beth murphy. the film follows susan retik and patti quigley, two women who lost their husbands on september 11, 2001, as they set up humanitarian programs for war widows in afghanistan. it premiered at the 2007 tribeca film festival.'}

In [7]:
d4 = {}
wiki_name = 'Castro Street (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d4[wiki_name.split('(')[0]] = soup.find('p').text.replace("\'",'').lower().replace(r'[^0-9.]+', '').split('\n')[0]
d4

{'Castro Street ': 'castro street (1966) is a visual nonstory documentary film which inspired by satie[2] uses the sounds and sights of a city street—in this case, castro street near the standard oil refinery in richmond, california complete with diesel trains and gas plants[3]—to convey the streets own mood and feel. there is no dialogue in this non-narrative experimental film. it was directed by bruce baillie.'}

In [8]:
d5 = {}
wiki_name = 'Finders Keepers (2015 film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d5[wiki_name.split('(')[0]] = soup.find('p').text.replace('\n','').replace("\''",'').lower().replace(r'[^0-9.]+', '').split(']')[1]
d5

{'Finders Keepers ': " the story details john wood's attempts to recover his mummified leg from shannon whisnant,\xa0after whisnant\xa0found the leg in a grill purchased at a storage unit auction."}

In [9]:
d6 = {}
wiki_name = 'Million Calorie March: The Movie (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d6[wiki_name.split('(')[0]] = soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d6      

{'Million Calorie March: The Movie ': 'million calorie march: the movie documents the million calorie march awareness and fundraising campaign and incorporates stories from marino’s 2005 book, "big & tall chronicles: misadventures of a lifelong food addict."  through flashbacks to his childhood, marino documents the influences that led to his struggles with weight loss and his personal battle to overcome food addiction. the purpose behind marino’s film is the increasing awareness of the spread of obesity throughout the united states and inspiring people to dig deep to fix themselves.'}

In [10]:
d7 = {}
wiki_name = 'No Place Like Home (2006 film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        #print(i)
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d7[wiki_name.split('(')[0]] = soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '').split('[')[0]
d7

{'No Place Like Home ': 'no place like home features a moving soundtrack made by musicians personally connected with the subjects and interviews with friends who survived katrina in new orleans. all proceeds from dvd sales go directly towards gulf coast restoration funds.'}

In [11]:
d8 = {}
wiki_name = 'The Outsider (2005 film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d8[wiki_name.split('(')[0]] = soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d8

{'The Outsider ': 'subtitled "a film about james toback," the outsider alternates among on-location shots, interviews with toback and with many of his film collaborators and celebrity friends, and commentary by film critic roger ebert. jarecki follows toback into the editing room after the initial shooting of when will i be loved. for this film, toback says he "shot freely," so that "all sorts of options are now open." toback describes film editing as a creative process on a par with acting or writing. "editing is the making of the movie." screenwriter and director robert towne accepts tobacks invitation to a screening of the first production cut of when will i be loved. accepting townes critique that a principal character in the film lacks adequate introduction, toback then orders a re-shoot, adding two scenes to the final cut he shops to distributors.'}

In [12]:
d9 = {}
wiki_name = 'Rolling Papers (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
desc = soup.find('p').text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d9[wiki_name.split('(')[0]] = desc
d9

{'Rolling Papers ': 'rolling papers is a 2015 documentary film directed by mitch dickman and featuring ricardo baca.[1][2][3] the "deceptively rote title" is said to be a winking reference to bacas work as a newspaper writer.[4]'}

In [13]:
d10 = {}
wiki_name = 'Surplus Terrorized into Being Consumers (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
d10[wiki_name.split('(')[0]] = soup.find('p').text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')
d10

{'Surplus Terrorized into Being Consumers ': 'surplus: terrorized into being consumers is an award-winning 2003 swedish documentary film on consumerism and globalization, created by director erik gandini and editor johan söderberg. it looks at the arguments for capitalism and technology, such as greater efficiency, more time and less work, and argues that these are not being fulfilled, and they never will be. the film leans towards anarcho-primitivist ideology and argues for a simple and fulfilling life.'}

In [14]:
d = {}
d.update(d1)
d.update(d2)
d.update(d3)
d.update(d4)
d.update(d5)
d.update(d6)
d.update(d7)
d.update(d8)
d.update(d9)
d.update(d10)
print(d)

{'Amarillo by Morning ': 'amarillo by morning is a 1997 documentary short by spike jonze. while filming pro bullriders for a commercial at the national rodeo in the houston astrodome in houston, texas, spike befriended two suburban teenagers who aspired to be cowboys. the documentary chronicles an afternoon in their lives.', 'Audience of One ': 'the film follows the story of a san francisco pentecostal minister richard gazowsky on his quest to shoot a groundbreaking fantasy film called gravity: the shadow of joseph (described by him as "star wars meets the ten commandments"). the film follows him and members of his church as they go through preproduction and fly to alberobello, italy, for initial shooting that turns out to be marred with difficulties. after returning home, gazowsky manages to arrange a lease of the treasure island film studio, but as their promised financing from german investors never materializes, they get evicted and eventually sued by the city of san francisco for 

In [15]:
doc_list = {}

wiki_name = 'Call + Response (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 2, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
        doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[0].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '') + \
        soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '') + \
        soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')


wiki_name = 'Midnight Movies: From the Margin to the Mainstream (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
    doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[0].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '') + \
        soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '') + \
        soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')


wiki_name = 'Operation Wedding (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
    doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')


wiki_name = 'Primary (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
    doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')

wiki_name = 'Poetry in Motion (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
    doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[1].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')    

wiki_name = 'The Roosevelts (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
        doc_list[wiki_name.split('(')[0]] = \
        soup.find('p').text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')


wiki_name = 'The Red String (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
        doc_list[wiki_name.split('(')[0]] = \
        soup.find('p').text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')

        
wiki_name = 'South of the Border (film) wiki'
for i in search(wiki_name, tld = 'com', lang = 'en', num = 1, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        soup = BeautifulSoup(requests.get(i).text, 'lxml')
        doc_list[wiki_name.split('(')[0]] = \
        soup.findAll('p')[2].text.replace('\n','').replace("\'",'').lower().replace(r'[^0-9.]+', '')




#print(len(doc_list.keys()))

doc_list.update(d)

df = pd.DataFrame(list(doc_list.items()),  columns = ['Title', 'Description'])
df.to_csv('doc_list/list_missing.csv', header = True, index = False)


In [ ]:
doc_list = {}
rem_list = {}

tab = list_table[0].findAll('i')   
for i in range(len(tab)):
    for node in tab[i].findAll('a'):
        if ''.join(node.findAll(text = True)).startswith('The'):
            name = ''.join(node.findAll(text = True))
            #links.append('https://www.wikipedia.org'+node.get('href'))

            p = wikipedia.page(name).sections[0]
            if p in all_possibles:
                doc_list[name] = wikipedia.page(name).section(p).replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
            else:
                doc_list[name] = wikipedia.page(name).summary.replace('\n','').replace("\'","").lower().replace(r'[^0-9.]+', '')
                
rem_list.update(doc_list)

In [ ]:
df = pd.DataFrame(list(rem_list.items()),  columns = ['Title', 'Description'])
df.to_csv('doc_list/list_rem.csv', header = True, index = False)

In [ ]:
whole_set = pd.read_csv('ml-20m/movies.csv')
whole_set.loc[whole_set['genres'] == 'Documentary']['title']
d = []
for i in whole_set.loc[whole_set['genres'] == 'Documentary']['title']:
    d.append(i.split('(')[0])
#print(d)

In [ ]:
query = 'Trials of Henry Kissinger, The _film'
for i in search(query, tld = 'com', lang = 'en', num = 2, start = 0, stop = 1, pause = 2):
    if 'https://en.wikipedia.org/wiki/' in i:
        print(i)